In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

SEED = 41
np.random.seed(SEED)

In [6]:
!ls ../data/01_raw

DataDictionaryWiDS2021.csv   TrainingWiDS2021.csv
SampleSubmissionWiDS2021.csv UnlabeledWiDS2021.csv
SolutionTemplateWiDS2021.csv


In [12]:
%%time
train = pd.read_csv('../data/01_raw/TrainingWiDS2021.csv')
test  = pd.read_csv('../data/01_raw/UnlabeledWiDS2021.csv')

CPU times: user 1.62 s, sys: 217 ms, total: 1.84 s
Wall time: 1.84 s


In [11]:
train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Data columns (total 181 columns):
 #    Column                       Non-Null Count   Dtype  
---   ------                       --------------   -----  
 0    Unnamed: 0                   130157 non-null  int64  
 1    encounter_id                 130157 non-null  int64  
 2    hospital_id                  130157 non-null  int64  
 3    age                          125169 non-null  float64
 4    bmi                          125667 non-null  float64
 5    elective_surgery             130157 non-null  int64  
 6    ethnicity                    128570 non-null  object 
 7    gender                       130091 non-null  object 
 8    height                       128080 non-null  float64
 9    hospital_admit_source        96959 non-null   object 
 10   icu_admit_source             129917 non-null  object 
 11   icu_id                       130157 non-null  int64  
 12   icu_stay_type                130157 non-nu

<ipython-input-11-4ab921678a25>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train.info(verbose=True, null_counts=True)


In [14]:
test.select_dtypes(include='object')

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type
0,Caucasian,F,Floor,Accident & Emergency,admit,Med-Surg ICU
1,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
2,Caucasian,F,Floor,Floor,admit,Med-Surg ICU
3,Caucasian,M,Floor,Floor,admit,Med-Surg ICU
4,Caucasian,M,Direct Admit,Accident & Emergency,admit,Med-Surg ICU
...,...,...,...,...,...,...
10229,Caucasian,F,Floor,Floor,admit,Med-Surg ICU
10230,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
10231,Caucasian,F,Step-Down Unit (SDU),Floor,admit,Med-Surg ICU
10232,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU


In [35]:
pd.factorize(train['icu_type'].values, sort=True)[1]

array(['CCU-CTICU', 'CSICU', 'CTICU', 'Cardiac ICU', 'MICU',
       'Med-Surg ICU', 'Neuro ICU', 'SICU'], dtype=object)

In [36]:
pd.factorize(test['icu_type'].values, sort=True)[1]

array(['CCU-CTICU', 'CSICU', 'CTICU', 'Cardiac ICU', 'MICU',
       'Med-Surg ICU', 'Neuro ICU', 'SICU'], dtype=object)

In [37]:
train['icu_type'][:10]

0           CTICU
1    Med-Surg ICU
2    Med-Surg ICU
3           CTICU
4    Med-Surg ICU
5    Med-Surg ICU
6    Med-Surg ICU
7    Med-Surg ICU
8       CCU-CTICU
9       CCU-CTICU
Name: icu_type, dtype: object

In [41]:
pd.factorize(train['icu_type'], sort=True)[0][:10]

array([2, 5, 5, 2, 5, 5, 5, 5, 0, 0])

In [39]:
test['icu_type'][:10]

0    Med-Surg ICU
1    Med-Surg ICU
2    Med-Surg ICU
3    Med-Surg ICU
4    Med-Surg ICU
5    Med-Surg ICU
6    Med-Surg ICU
7    Med-Surg ICU
8    Med-Surg ICU
9    Med-Surg ICU
Name: icu_type, dtype: object

In [42]:
pd.factorize(test['icu_type'], sort=True)[0][:10]

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [45]:
train.drop('Unnamed: 0', axis=1)

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
3,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,...,337.0,337.0,0,0,0,0,0,0,0,0
4,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130152,164758,7,50.0,29.287256,0,Caucasian,M,175.3,Emergency Department,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
130153,197653,7,79.0,29.653433,0,Caucasian,F,162.6,Direct Admit,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
130154,219719,7,73.0,32.265371,0,African American,M,177.8,Emergency Department,Accident & Emergency,...,163.0,163.0,0,0,0,0,0,0,0,1
130155,222562,170,81.0,24.408579,0,Caucasian,M,185.4,Emergency Department,Accident & Emergency,...,NaN,NaN,0,0,0,0,0,0,0,0
